In [8]:
! pip install captum --no-deps

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from captum.attr import LayerIntegratedGradients, visualization as viz
import pandas as pd

In [9]:
checkpoint = '/kaggle/input/datasets/dmytriypan/distilbert-classifier-checkpoints/output/distilbert/distilbert-base-multilingual-cased_llm_trace_classification/checkpoint-6430'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [10]:
def forward_func(input_ids, attention_mask):
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    probs = F.softmax(outputs.logits, dim=-1)
    return probs[:, 0]  # вероятность класса 0 (AI)

In [11]:
lig = LayerIntegratedGradients(forward_func, model.distilbert.embeddings)

In [12]:
def interpret_text(text, lig=lig):
    inputs = tokenizer(text, max_length=256, truncation=True, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    attributions, delta = lig.attribute(
        inputs=input_ids,
        additional_forward_args=(attention_mask,),
        return_convergence_delta=True
    )

    attributions = attributions.sum(dim=-1).squeeze(0).detach().cpu()
    tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze(0))
    attributions = attributions / torch.norm(attributions)

    vis_data = viz.VisualizationDataRecord(
        attributions,
        torch.max(F.softmax(model(input_ids, attention_mask).logits, dim=-1)),
        torch.argmax(model(input_ids, attention_mask).logits),
        torch.argmax(model(input_ids, attention_mask).logits),
        text,
        attributions.sum(),
        tokens,
        delta
    )

    # Создаем DataFrame для токенов и их важности
    df_tokens = pd.DataFrame({'token': tokens, 'attribution': attributions.numpy()})
    return vis_data, df_tokens

In [13]:
def interpret_texts_batch(texts):
    visualizations = []
    dfs = []
    for text in texts:
        vis_data, df_tokens = interpret_text(text)
        visualizations.append(vis_data)
        dfs.append(df_tokens)
    return visualizations, dfs

In [23]:
import pandas as pd

splits = {'train': 'train.jsonl', 'validation': 'valid.jsonl', 'test': 'test.jsonl'}
df = pd.read_json("hf://datasets/iitolstykh/LLMTrace_classification/" + splits["test"], lines=True)

In [32]:
df_ai_ru = df.loc[(df['lang'] == 'ru') & (df['label']=='ai')].sample(20, random_state=42)
df_ai_eng = df.loc[(df['lang'] == 'eng') & (df['label']=='ai')].sample(20, random_state=42)

df_ai = pd.concat([df_ai_ru, df_ai_eng], axis = 0)
df_ai

,lang,label,model,data_type,prompt_type,topic_id,text,prompt
2568,ru,ai,yandex/YandexGPT-5-Lite-8B-instruct,article,delete,6c18c0b121a08440f2d8b514efa7f42d,**Отчёт о дегустации**\n\n**Интерьер**\n\nИнте...,Сожми этот длинный обзор до лаконичной статьи ...
36978,ru,ai,gpt-3.5,question,create,a84a2c4323c197fdcf8fe4e23566216e,Выплата страховой части государственной пенсии...,Какие особенности выплаты страховой части госу...
4915,ru,ai,gpt-4-1106-preview,poetry,update,2530a741c86d6bd36643d66460b97327,"И вот взошло сияющее Солнце, свет его раскинул...",Перепиши текст в стиле Булгакова. Можешь модиф...
18613,ru,ai,google/gemma-2-27b-it,article,delete,25ee17d72ac475e4f95c90df3dfcaf02,6 октября в рамках IT-фестиваля BitByte пройдё...,"Урежь этот объёмный текст до 60–100 слов, сохр..."
44708,ru,ai,google/gemma-2-27b-it,short_form,update,7079930e14ea14bc3aef5a1769da3bcb,Сердце партии «Великое Отечество» переполнено ...,"Переформулируй этот пост, применив более образ..."
30793,ru,ai,gpt-4,question,create,8e530fa5555d39bdd6249330de565ea2,Какие ключевые элементы на этой картинке? Изоб...,What are the key elements in this picture? T...
30490,ru,ai,gpt-4-0125-preview,question,update,ad2a4a42fac84acd261f34881fb267ee,В стопах Felis catus (домашних кошек) присутст...,Перепиши текст в научном стиле. Можешь модифиц...
23229,ru,ai,WizardLM-2-7B,poetry,create,ab01d1eeb1c1311503b24f987d3673a7,"Закат пылает алым светом, И облака как будто б...",Продолжи текст стихотворения в жанре лирика (п...
46175,ru,ai,o3-2025-04-16,factual,delete,5fdd2216acb2c881b3899bf958853e4b,"Во время одной весенней прогулки автор, затяну...",Убери из этого литературного произведения всё ...
49881,ru,ai,CohereForAI/c4ai-command-r-08-2024,article,update,d6456fa343f615b5c0367af8402a97d7,Взлом сайта в ночное время стал настоящим шоко...,"Отредактируй язык этой статьи, используя более..."


In [36]:
texts_AI = df_ai['text'].to_list()

visualizations, dfs = interpret_texts_batch(texts_AI)

# Показываем визуализацию всех текстов
viz.visualize_text(visualizations)

# Выводим DataFrame для первого текста
print(dfs[0])

      token  attribution
0     [CLS]     0.513396
1         *     0.114037
2         *     0.073765
3        От     0.018969
4     ##чёт     0.034211
..      ...          ...
251    двух    -0.042207
252       к    -0.060843
253  ##уска    -0.106350
254     ##х    -0.047427
255   [SEP]     0.507466

[256 rows x 2 columns]
